**Respondendo às perguntas formuladas no objetivo**
- Pergunta 1 - Quais o Top10 aplicativos com maior número de avaliações? - app x reviews
- Pergunta 2 - Quais o Top10 aplicativos com a melhor avaliação? - app x rating
- Pergunta 3 - Qual o categoria com a melhor avaliação média? - category x rating
- Pergunta 7 - Quais os app com as piores avaliações? - app x rating
- (?)Existe uma correlação entre o número de instalações e avaliação média? - installs x rating
- Quais categorias são as mais populares (> número de instalações)? - category x installs
- Qual o preço médio dos aplicativos pagos? - cost_type x price
- Qual a proporção entre aplicativos com instalação paga e gratuita? cost_type

In [0]:

%sql
DROP CATALOG IF EXISTS mvp CASCADE

In [0]:

%sql
CREATE CATALOG mvp

In [0]:

%sql
USE CATALOG mvp

In [0]:

%sql
DROP DATABASE IF EXISTS bronze CASCADE;
DROP DATABASE IF EXISTS silver CASCADE;
DROP DATABASE IF EXISTS gold CASCADE;

In [0]:

%sql
CREATE SCHEMA IF NOT EXISTS bronze COMMENT 'Camada para dados brutos ingeridos';
CREATE SCHEMA IF NOT EXISTS silver COMMENT 'Camada para dados limpos, filtrados e enriquecidos';
CREATE SCHEMA IF NOT EXISTS gold COMMENT 'Camada para dados agregados e prontos para análise';

Camada Bronze

In [0]:

%sql
USE SCHEMA bronze

In [0]:
%sql
CREATE VOLUME bronze

**Localização original do dataset:**
L. Gupta, "Google Play Store Apps," Feb 2019. [Online]. Available: 
https://www.kaggle.com/lava18/google-play-store-apps
**License**
CC BY-SA 4.0: Livre para 
Compartilhar — copie e redistribua o material em qualquer meio ou formato para qualquer finalidade, inclusive comercial.
Adaptar — remixar, transformar e desenvolver o material para qualquer finalidade, inclusive comercial.

Devido a restrições de segurança do computador utilizado, não foi possível acessar diretamente o Kaggle, portanto o dataset foi salvo no repositório do MVP no Github e acessado por este caminho.

In [0]:
#Download do dataset dos aplicativos do Google Play
dbutils.fs.cp(
    "https://github.com/Romario76/Pos-PUCRJ-Data-Science-Analytics/raw/refs/heads/SPRINT_EngDADOS/googleplaystore.csv", 
    "dbfs:/Volumes/mvp/bronze/bronze/googleplaystore.csv")

True

In [0]:
#Definição do catálogo e do schema para ingestão dos dados
spark.sql("USE CATALOG mvp")
spark.sql("USE SCHEMA bronze")

DataFrame[]

In [0]:
from pyspark.sql.functions import col

# Função para normalizar nomes
def normalize_columns(df):
    new_cols = [c.lower().replace(" ", "_") for c in df.columns]
    return df.toDF(*new_cols)

In [0]:
#Conversão do arquivo googleplaystore.csv em um Dataframe do Spark
df_appBase = spark.read.option("header", True).option("inferSchema", True).csv("dbfs:/Volumes/mvp/bronze/bronze/googleplaystore.csv")

#Renomeação das colunas do Dataframe com caracteres inválidos
df_appBase_normalized = normalize_columns(df_appBase)

display(df_appBase_normalized.limit(10))



app,category,rating,reviews,size,installs,type,price,content_rating,genres,last_updated,current_ver,android_ver
Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
"U Launcher Lite – FREE Live Cool Themes, Hide Apps",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6M,"50,000+",Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up
Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19M,"50,000+",Free,0,Everyone,Art & Design,"April 26, 2018",1.1,4.0.3 and up
Infinite Painter,ART_AND_DESIGN,4.1,36815,29M,"1,000,000+",Free,0,Everyone,Art & Design,"June 14, 2018",6.1.61.1,4.2 and up
Garden Coloring Book,ART_AND_DESIGN,4.4,13791,33M,"1,000,000+",Free,0,Everyone,Art & Design,"September 20, 2017",2.9.2,3.0 and up
Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121,3.1M,"10,000+",Free,0,Everyone,Art & Design;Creativity,"July 3, 2018",2.8,4.0.3 and up


In [0]:
#Criação da tabela app_base no formato Delta na camada bronze
df_appBase_normalized.write.format("delta").mode("overwrite").saveAsTable("app_base")

In [0]:
%sql
SELECT * FROM mvp.bronze.app_base
LIMIT 10;

app,category,rating,reviews,size,installs,type,price,content_rating,genres,last_updated,current_ver,android_ver
Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
"U Launcher Lite – FREE Live Cool Themes, Hide Apps",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6M,"50,000+",Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up
Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19M,"50,000+",Free,0,Everyone,Art & Design,"April 26, 2018",1.1,4.0.3 and up
Infinite Painter,ART_AND_DESIGN,4.1,36815,29M,"1,000,000+",Free,0,Everyone,Art & Design,"June 14, 2018",6.1.61.1,4.2 and up
Garden Coloring Book,ART_AND_DESIGN,4.4,13791,33M,"1,000,000+",Free,0,Everyone,Art & Design,"September 20, 2017",2.9.2,3.0 and up
Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121,3.1M,"10,000+",Free,0,Everyone,Art & Design;Creativity,"July 3, 2018",2.8,4.0.3 and up


In [0]:
%sql
DESCRIBE TABLE EXTENDED mvp.bronze.app_base

col_name,data_type,comment
app,string,null
category,string,null
rating,string,null
reviews,string,null
size,string,null
installs,string,null
type,string,null
price,string,null
content_rating,string,null
genres,string,null


Camada Silver

In [0]:
spark.sql("USE CATALOG mvp")
spark.sql("USE SCHEMA silver")

DataFrame[]

Inicialmente, ia realizar os tratamentos em uma Dataframe Spark. No momento de realizar as conversões dos tipos dos dados (int, double), ocorreram erros causados por um registro 'corrompido': é provável que na extração do dataset, o campo "category" de um registro não foi identificado e, ao invés de ter sido gravado como null, o campo foi ignorado, fazendo com que os valores dos demais campo fossem "deslocados para esquerda" no momento da conversão na camada bronze.

In [0]:
%sql
DROP TABLE silver.app_base_silver

In [0]:
%sql
CREATE OR REPLACE TABLE silver.app_base_silver
USING delta
COMMENT 'Tabela contendo os dados do aplicativos Google Play da camada silver'
AS
SELECT
  app AS app_name,
  category,
  rating,
  reviews,
  CASE -- Limpeza do size
    WHEN size = 'Varies with device' THEN NULL
    ELSE size
  END AS size,
  price,
  content_rating,
  genres,
  regexp_replace(installs, '[+,]', '') AS installs, --limpeza de installs
  type AS cost_type -- Renomeando 'type' para evitar conflito
  -- Colunas 'last_updated', 'current_ver' e 'android_ver' são colunas dimensionais que não interferem nas análises propostas e não serão utilizadas
FROM bronze.app_base

num_affected_rows,num_inserted_rows


### **Qualidade de dados**
Identificação de valores nulos ou inválidos em relação ao tipo esperado após conversão

_Valores nulos por atributo_

In [0]:
%sql
SELECT 
    COUNT(CASE WHEN app_name IS NULL THEN 1 END) AS null_count_app_name,
    COUNT(CASE WHEN category IS NULL THEN 1 END) AS null_count_category,
    COUNT(CASE WHEN rating IS NULL THEN 1 END) AS null_count_rating,
    COUNT(CASE WHEN reviews IS NULL THEN 1 END) AS null_count_reviews,
    COUNT(CASE WHEN size IS NULL THEN 1 END) AS null_count_size,
    COUNT(CASE WHEN installs IS NULL THEN 1 END) AS null_count_installs,
    COUNT(CASE WHEN cost_type IS NULL THEN 1 END) AS null_count_cost_type,
    COUNT(CASE WHEN price IS NULL THEN 1 END) AS null_count_price,
    COUNT(CASE WHEN content_rating IS NULL THEN 1 END) AS null_count_content_rating,
    COUNT(CASE WHEN genres IS NULL THEN 1 END) AS null_count_genres
FROM silver.app_base_silver;

null_count_app_name,null_count_category,null_count_rating,null_count_reviews,null_count_size,null_count_installs,null_count_cost_type,null_count_price,null_count_content_rating,null_count_genres
0,0,0,0,1694,0,0,0,1,0


Na criação da tabela na camada 'silver', valores de texto do atributo 'size' foi substituído por 'null', explicando o alto número de nulos.
Portanto, originalmente, apenas um atributo ('content_rating') contém valor nulo (uma linha).
A estratégia para tratamento dos nulos será:
'content_rating': exclusão da linha;
'size': popular com a média dos demais valores, após conversão dos literais com 'M' e 'k'.

In [0]:
%sql
DELETE FROM silver.app_base_silver
WHERE content_rating IS NULL

num_affected_rows
1


In [0]:
%sql
SELECT 
    COUNT(content_rating) AS null_count_content_rating
FROM silver.app_base_silver
WHERE content_rating IS NULL

null_count_content_rating
0


Verificação de dados inválidos
Os tipos dos dados das colunas 'rating', 'reviews', 'installs', 'price' e 'size' sugerem precisar serem ajustadas para tipos númericos (int ou double).
Antes de realizar as conversões, será avaliada a existência de valores inválidos e os devidos tratamentos, para cada atributo.

In [0]:
%sql
SELECT --Teste de validação se não há dados inválidos (textos, símbolos etc)
    SUM(rating) AS invalid_count_rating
FROM silver.app_base_silver
WHERE NOT regexp_like(rating, '^[+-]?[0-9]+(\\.[0-9]+)?$');

invalid_count_rating
1476


Visualização dos dados inválidos para cada atributo, afim de definir a melhor estratégia de limpeza (exclusão ou substituição)

In [0]:
%sql
SELECT rating, COUNT(*)
FROM silver.app_base_silver
WHERE NOT regexp_like(rating, '^[+-]?[0-9]+(\\.[0-9]+)?$')
GROUP BY rating
    

rating,COUNT(*)


Ocorrência de três grupos de dados inválidos.
Estratégia para as strings 'navigation' e 'Body': Exclusão do registro, devido ao baixo impacto.
Estratégia para 'NaN': inserir a média dos demais valores do atributo.

In [0]:
%sql
DELETE FROM silver.app_base_silver
WHERE rating == ' navigation' OR rating == 'Body'

num_affected_rows
2


In [0]:
%sql
-- Cálculo da média de rating (excluindo 'NaN')
WITH avg_rating_cte AS (
  SELECT AVG(CAST(REGEXP_REPLACE(rating, '[^0-9.]', '') AS DOUBLE)) AS avg_rating
  FROM silver.app_base_silver
  WHERE rating != 'NaN'
)

-- Atualizando as linhas onde rating é 'NaN' com a média calculada
UPDATE silver.app_base_silver
SET rating = (SELECT avg_rating FROM avg_rating_cte)
WHERE rating == 'NaN';

num_affected_rows
1474


In [0]:
%sql
SELECT COUNT(rating)
FROM silver.app_base_silver
WHERE NOT regexp_like(rating, '^[+-]?[0-9]+(\\.[0-9]+)?$');

COUNT(rating)
0


In [0]:
%sql
SELECT --Teste de validação se não há dados inválidos (textos, símbolos etc)
    COUNT(reviews) AS invalid_count_reviews
FROM silver.app_base_silver
WHERE NOT regexp_like(reviews, '^[+-]?[0-9]+(\\.[0-9]+)?$');

invalid_count_reviews
0


In [0]:
%sql
SELECT --Teste de validação se não há dados inválidos (textos, símbolos etc)
    COUNT(size) AS invalid_count_size
FROM silver.app_base_silver
WHERE NOT regexp_like(size, '^[+-]?[0-9]+(\\.[0-9]+)?$');

invalid_count_size
9144


In [0]:
%sql
SELECT size, COUNT(*) AS invalid_count_size
FROM silver.app_base_silver
WHERE NOT regexp_like(size, '^[+-]?[0-9]+(\\.[0-9]+)?$')
GROUP BY size
ORDER BY count(*) DESC
LIMIT 15;


size,invalid_count_size


As linhas consideradas inválidas possuem as siglas 'k' (kilo) e 'M' (mega).
Estratégia: eliminar as siglas e colocar em uma mesma base (bytes).
Para os valores literais 'none', será realizada em duas etapas: a primeira altera o valor para null, a segunda substitui os nulos pela médias dos valores válidos de tamanho dos aplicativos do dataset.

In [0]:
%sql
-- Converte strings como '10M', '512K', '1G' para kB(kilobytes) (base 1024)
-- try_cast retorna null para valores inválidos
-- regexp_replace remove todos os caracteres não numéricos
UPDATE silver.app_base_silver
SET size = try_cast(regexp_replace(size, '[^0-9.]', '') AS DOUBLE) *
  CASE
    --WHEN regexp_like(size, '.*[Kk]$') THEN POW(1024, 1)
    WHEN regexp_like(size, '.*[Mm]$') THEN POW(1024, 1)
    WHEN regexp_like(size, '.*[Gg]$') THEN POW(1024, 2)
    WHEN regexp_like(size, '.*[Tt]$') THEN POW(1024, 3)
    ELSE 1
  END


num_affected_rows
10838


In [0]:
%sql
-- Cálculo da média de size (excluindo 'null')
WITH avg_size_cte AS (
  SELECT AVG(size) AS avg_size
  FROM silver.app_base_silver
  WHERE size IS NOT NULL
)

-- Atualizando as linhas NULL com a média calculada
UPDATE silver.app_base_silver
SET size = (SELECT avg_size FROM avg_size_cte)
WHERE size IS NULL;

num_affected_rows
1694


In [0]:
%sql
SELECT --Teste de validação se não há dados inválidos (textos, símbolos etc)
    COUNT(size) AS invalid_count_size
FROM silver.app_base_silver
WHERE NOT regexp_like(size, '^[+-]?[0-9]+(\\.[0-9]+)?$');

invalid_count_size
0


In [0]:
%sql
SELECT --Teste de validação se não há dados inválidos (textos, símbolos etc)
    COUNT(installs) AS invalid_count_installs
FROM silver.app_base_silver
WHERE NOT regexp_like(installs, '^[+-]?[0-9]+(\\.[0-9]+)?$');

invalid_count_installs
0


In [0]:
%sql
SELECT --Teste de validação se não há dados inválidos (textos, símbolos etc)
    COUNT(price) AS invalid_count_price
FROM silver.app_base_silver
WHERE NOT regexp_like(price, '^[+-]?[0-9]+(\\.[0-9]+)?$');

invalid_count_price
800


In [0]:
%sql
SELECT price, count(price) AS invalid_count_price
FROM silver.app_base_silver
WHERE NOT regexp_like(price, '^[+-]?[0-9]+(\\.[0-9]+)?$')
GROUP BY price
ORDER BY count(*) DESC

price,invalid_count_price
$0.99,148
$2.99,129
$1.99,73
$4.99,72
$3.99,63
$1.49,46
$5.99,30
$2.49,26
$9.99,21
$6.99,13


In [0]:
%sql
UPDATE silver.app_base_silver
SET price = replace(price, '$', '')

num_affected_rows
10838


In [0]:
%sql
SELECT count(price) AS invalid_count_price
FROM silver.app_base_silver
WHERE NOT regexp_like(price, '^[+-]?[0-9]+(\\.[0-9]+)?$')


invalid_count_price
0


Todas as limpezas realizadas mantiveram os tipos de dados originais ("string").
Para realizar a alteração, utilizei o PySpark, carregando os dados da tabela em um dataframe, alterando os tipos de dados dos atributos necessários e, por fim, sobreescrevendo o schema da tabela no banco.
Na minha opinião, é uma solução mais simples e com menos processamento, pois as alternativas seriam, ou replicar os dados em uma nova tabela, com o schema desejado, descartando a original e renomeando a nova, ou realizar operação semelhante, porém com cada coluna/atributo na própria tabela.

In [0]:

from pyspark.sql import functions as F

table = "silver.app_base_silver"

df_Str = spark.table(table)

# limpar não-numéricos; mantém NULL para falhas
df_Num = df_Str.withColumn("rating",F.col("rating").cast("double")) \
    .withColumn("reviews",F.col("reviews").cast("long")) \
    .withColumn("size",F.col("size").cast("double")) \
    .withColumn("installs",F.col("installs").cast("long")) \
    .withColumn("price",F.col("price").cast("double"))

# Sobrescrever usando Delta, preservando nome da tabela
(df_Num
 .write
 .format("delta")
 .mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable(table))


A fonte dos dados indica a existência de duplicidades no atributo app_name. Como não há outro atributo que identifica unicamente cada registro, app_name será considerado como este identificador.



Adicionando comentários no metadados da tabela com a descrição detalhada da colunas.

In [0]:
appBase_silver_comments = {
    "app_name": "Nome do aplicativo",
    "category": "Categoria do aplicativo",
    "rating": "Avaliação geral dos usuários para cada aplicativo",
    "reviews": "Número total de avaliações do aplicativo",
    "size": "Tamanho do aplicativo em kB",
    "price": "Preço do aplicativo em dólares",
    "content_rating": "Classificação indicativa do aplicativo",
    "genres": "Gêneros associados ao aplicativo",
    "installs": "Número total de downloads/instalações do aplicativo",
    "costs_type": "Indica se o aplicativo é gratuito ou pago"
}

for col_name, comment in appBase_silver_comments.items():
    try:
        spark.sql(f"ALTER TABLE {table} ALTER COLUMN {col_name} COMMENT '{comment}'")
    except Exception as e:
        print(f"  Aviso: Não foi possível aplicar comentário para a coluna '{col_name}'. Erro: {e}")

print("\n✅ Comentários aplicados às colunas das tabelas Silver.")

  Aviso: Não foi possível aplicar comentário para a coluna 'costs_type'. Erro: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `costs_type` cannot be resolved. Did you mean one of the following? [`app_name`, `category`, `rating`, `reviews`, `size`, `price`, `content_rating`, `genres`, `installs`, `cost_type`]. SQLSTATE: 42703; line 1 pos 0

JVM stacktrace:
org.apache.spark.sql.AnalysisException
	at org.apache.spark.sql.errors.QueryCompilationErrors$.unresolvedColumnError(QueryCompilationErrors.scala:4586)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveFieldNameAndPosition$.$anonfun$resolveFieldNames$1(Analyzer.scala:5578)
	at scala.Option.getOrElse(Option.scala:201)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveFieldNameAndPosition$.org$apache$spark$sql$catalyst$analysis$Analyzer$ResolveFieldNameAndPosition$$resolveFieldNames(Analyzer.scala:5577)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveFieldNameAndPosit

In [0]:
%sql
DESCRIBE TABLE silver.app_base_silver

col_name,data_type,comment
app_name,string,Nome do aplicativo
category,string,Categoria do aplicativo
rating,double,Avaliação geral dos usuários para cada aplicativo
reviews,bigint,Número total de avaliações do aplicativo
size,double,Tamanho do aplicativo em kB
price,double,Preço do aplicativo em dólares
content_rating,string,Classificação indicativa do aplicativo
genres,string,Gêneros associados ao aplicativo
installs,bigint,Número total de downloads/instalações do aplicativo
cost_type,string,null


## Camada gold

Pergunta 1 - Quais o Top10 aplicativos com maior número de avaliações? - app x reviews
Pergunta 2 - Quais o Top10 aplicativos com a melhor avaliação? - app x rating
Pergunta 3 - Qual o categoria com a melhor avaliação média? - category x rating
Pergunta 7 - Quais os app com as piores avaliações? - app x rating
(?)Existe uma correlação entre o número de instalações e avaliação média? - installs x rating
Quais categorias são as mais populares (> número de instalações)? - category x installs
Qual o preço médio dos aplicativos pagos? - cost_type x price
Qual a proporção entre aplicativos com instalação paga e gratuita? cost_type

In [0]:
%sql
Select app_name, count(app_name) as count_app
from silver.app_base_silver
group by app_name
order by count_app desc

app_name,count_app
ROBLOX,9
"CBS Sports App - Scores, News, Stats & Watch Live",8
ESPN,7
8 Ball Pool,7
Candy Crush Saga,7
Duolingo: Learn Languages Free,7
slither.io,6
Helix Jump,6
Temple Run 2,6
Bubble Shooter,6


In [0]:
%sql
SELECT * FROM silver.app_base_silver
WHERE app_name = 'CBS Sports App - Scores, News, Stats & Watch Live'
    

app_name,category,rating,reviews,size,price,content_rating,genres,installs,costs_type
"CBS Sports App - Scores, News, Stats & Watch Live",SPORTS,4.3,91031,22034.402985564328,0.0,Everyone,Sports,5000000,Free
"CBS Sports App - Scores, News, Stats & Watch Live",SPORTS,4.3,91031,22034.402985564328,0.0,Everyone,Sports,5000000,Free
"CBS Sports App - Scores, News, Stats & Watch Live",SPORTS,4.3,91031,22034.402985564328,0.0,Everyone,Sports,5000000,Free
"CBS Sports App - Scores, News, Stats & Watch Live",SPORTS,4.3,91031,22034.402985564328,0.0,Everyone,Sports,5000000,Free
"CBS Sports App - Scores, News, Stats & Watch Live",SPORTS,4.3,91033,22034.402985564328,0.0,Everyone,Sports,5000000,Free
"CBS Sports App - Scores, News, Stats & Watch Live",SPORTS,4.3,91033,22034.402985564328,0.0,Everyone,Sports,5000000,Free
"CBS Sports App - Scores, News, Stats & Watch Live",SPORTS,4.3,91033,22034.402985564328,0.0,Everyone,Sports,5000000,Free
"CBS Sports App - Scores, News, Stats & Watch Live",SPORTS,4.3,91035,22034.402985564328,0.0,Everyone,Sports,5000000,Free


In [0]:
%sql
--Quais o Top10 aplicativos com maior número de avaliações?
SELECT app_name, SUM(reviews) AS total_reviews
FROM silver.app_base_silver
GROUP BY app_name
ORDER BY total_reviews DESC
LIMIT 10

app_name,total_reviews
Instagram,266241989
WhatsApp Messenger,207348304
Clash of Clans,179558781
Messenger – Text and Video Chat for Free,169932272
Subway Surfers,166331958
Candy Crush Saga,156993136
Facebook,156286514
8 Ball Pool,99386198
Clash Royale,92530298
Snapchat,68045010
